# Multiple simulations of the MARM1 model

This notebook allows to run multiple sequential simulations of the MARM1 model in order to generate a dataset for multiple conditions. For a detailed description of how a single simulation run is set up and the visulization of a time-course trajectory please refer to the Jupyter Notebook: *MARM1_simulation_single_run.ipynb*. 

Importing the model and libraries necessary to run MARM1 model simulations. 

In [6]:
%matplotlib notebook
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
#load model 
from pysb.simulator import ScipyOdeSimulator
from pysb.core import as_complex_pattern
from MARM1 import model

Generating the condition settings (e.g. RAFi and MEKi concentrations) for each model simulations to be run.

In [7]:
#define conditions
n_doses=1;
RAFi_concentration = np.append(np.array([0.0]), np.logspace(-4, 1, n_doses))
MEKi_concentration = np.append(np.array([0.0]), np.logspace(-5, 0, n_doses))
t_pretrt = [24]
EGF_concentration = [0, 100]
t_trt = [2]
param_set_index = [0] #np.linspace(0,99,100, dtype=int);
N_time_points = 25
#generate condition settings for multiple simulations
settings_list=[]
for param in param_set_index:
    for rafi in RAFi_concentration:
        for meki in MEKi_concentration:
            for pretrt in t_pretrt:
                for trt in t_trt:
                    for egfc in EGF_concentration:
                        settings_list.append([param, pretrt, rafi, meki, trt, egfc])

Defining support functions to run simulations. 

In [8]:
def equilibrate(simulator, initials):
    """Simulate a model from given initial conditions until it reaches steady state"""
    scale = 10
    t_start = 1e-4
    df = None
    tspan = np.geomspace(t_start, t_start * scale)
    while True:
        #print(f"    at t={tspan[-1]:<5.3g} ... ", end='', flush=True)
        res = simulator.run(tspan=tspan, initials=initials)
        df = pd.concat([df, res.dataframe.iloc[1:]])
        initials = res.species[-1]
        close = np.isclose(
            *res.species[[-1,-2]].view(float).reshape(2,-1),
            rtol=1e-3
        )
        cs = np.sum(close)
        n = len(simulator.model.species)
        #print(f"{cs}/{n} species converged")
        if np.all(close):
            break
        tspan *= scale
    return df

In [9]:
def get_species_index(model, pattern):
    """Return the integer species number for a given species in the model"""
    pattern = as_complex_pattern(pattern)
    matches = [
        i for i, s in enumerate(model.species)
        if s.is_equivalent_to(pattern)
    ]
    n = len(matches)
    assert n == 1, f"Expected exactly one match, got {n}"
    return matches[0]

Simulating multiple condition runs and save the corresponding time-course trajectories on disk. 

In [ ]:
param_prev = -1
#run a simulation for each selected condition
for iset in tqdm_notebook(range(len(settings_list)), desc='Simulation progress'):
   
    #unload the settings and prepare the unperturbed model
    [param, pretrt, rafi, meki, trt, egfc] =  settings_list[iset]
    
    #run a simulation of the unperturbed model to obtain initial steady state (if not run before)
    if not (param == param_prev):
        param_sets = pd.read_csv('parameter_sets.csv', index_col=0)
        param_sets = param_sets.drop('chi2', axis=1)
        params = param_sets.iloc[param].to_dict()
        sim = ScipyOdeSimulator(model, param_values=params, atol=1E-50)
        df_eq = equilibrate(sim, None)
        

    #run a time-course simulation for the pretreatment phase
    RAFi_index = get_species_index(model, model.monomers.RAFi(raf=None))
    MEKi_index = get_species_index(model, model.monomers.MEKi(mek=None))
    EGF_index = get_species_index(model, model.monomers.EGF(rtk=None))
    initials_pre = df_eq.iloc[-1, :len(model.species)].copy()
    initials_pre[RAFi_index] = rafi
    initials_pre[MEKi_index] = meki
    initials_pre[EGF_index] = 0.0
    tspan_pretrt = np.linspace(0, pretrt, N_time_points)
    pre_sim= sim.run(tspan=tspan_pretrt, initials=initials_pre)
    df_pre_sim = pre_sim.dataframe
    df_pre = pd.concat([df_pre_sim.loc[:pretrt], df_pre_sim.iloc[[-1]]])
    df_pre['time'] = df_pre.index
    df_pre['time'] -= df_pre['time'].iloc[-2]
    df_pre['time'].iloc[-1] = 0
    
    #run a time-course simulation for the EGF perturbaion phase
    tspan_trt = np.linspace(0, trt, N_time_points)
    initials_trt = df_pre.iloc[-1, :len(model.species)].copy()
    initials_trt[EGF_index] = egfc
    res_trt = sim.run(tspan=tspan_trt, initials=initials_trt)
    
    #concatenate pretreatment and EGFR perturnations and settings 
    obs_names = [x.name for x in model.observables]
    obs = pd.concat([df_pre, res_trt.dataframe])[obs_names]
    obs.loc[:, (obs < 1e-10).all()] = 0
    
    #append simulations results on file 
    df_settings=pd.DataFrame(obs.shape[0]*[[param, pretrt, rafi, meki, trt, egfc]], columns=['param_set_index', 'time_pre_treatment' , 'RAFi_concentration' , 'MEKi_concentration' , 'time_treatment', 'EGF_concentration'] )
    obs = obs.join(df_settings.set_index(obs.index))
    #obs = pd.concat([df_settings,obs.index], axis=1)
    if (iset == 0): 
       obs.to_csv('trajectories_multiple_runs.csv', mode='w', header=True)
    else:
       obs.to_csv('trajectories_multiple_runs.csv', mode='a', header=False)
        
    #update param used
    param_prev=param